Final protocol 5/7/25

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from scipy.integrate import cumulative_trapezoid
import os
from scipy.optimize import curve_fit
from scipy.constants import N_A
from tqdm.auto import tqdm
from scipy.optimize import minimize_scalar

import statsmodels.api as sm


In [ ]:
AREA = {
    0 : 2 * 73.2e-10 * 74.3e-10,
    5 : 2 * 73.1e-10 * 74.4e-10,
    10 : 2 * 73.1e-10 * 74.3e-10,
    20 : 2 * 73.3e-10 * 74.0e-10,
    40 : 2 * 72.8e-10 * 73.9e-10,
    60 :  2 * 72.5e-10 * 73.7e-10,
    80 : 2 * 72.7e-10 * 73.9e-10,
}

In [ ]:
def read_Tgraph_chunks(base_remote_path, file_name, k):
    Temp_chunks = []
    with open(f'{base_remote_path}/slab_position_verify.txt', 'r') as file:
        for _ in range(3):
            next(file)
        line = next(file).strip().split()
        Nchunks_tot = int(line[1])
    with open(f'{base_remote_path}/{file_name}', 'r') as file:
        for i in range(k):
            if i == 0:
                # Skip the first 4 lines for the first chunk
                for _ in range(4):
                    next(file)
            else:
                # Skip the first 2 lines for subsequent chunks
                next(file)
            
            # Read the next N lines
            chunk = []
            for _ in range(Nchunks_tot):
                line = next(file).strip()
                values = list(map(float, line.split()))
                chunk.append(values)
            
            Temp_chunks.append(np.array(chunk))
    return Temp_chunks, Nchunks_tot

def process_chunks(Temp_chunks, Nchunks_tot, Nchunks, slab_skip=3): # Nchunks_tot: total chunks; Nchunks: target chunks for temp evaluatation
    k = len(Temp_chunks)
    T_slab_positive = np.zeros(k)
    T_slab_negative = np.zeros(k)
    T_bulk_chunk = np.zeros(k)

    # elem_transient=5000 # skip first 50 ps

    for i in range(k):
        up = Nchunks_tot // 2 + int(slab_skip) 
        down = Nchunks_tot // 2 - 1 - int(slab_skip) 
        
        # Average layers above graphene
        positive_layers = [Temp_chunks[i][up + j, 1] for j in range(Nchunks-slab_skip)]
        T_slab_positive[i] = np.mean(positive_layers)
        # if i==30000:
        #     print(positive_layers) # test
        
        # Average layers below graphene
        negative_layers = [Temp_chunks[i][down - j, 1] for j in range(Nchunks-slab_skip)]
        T_slab_negative[i] = np.mean(negative_layers)
        # if i==30000:
        #     plt.plot(negative_layers)

        # Average temperature across the entire chunk
        T_bulk_chunk[i] = np.mean(Temp_chunks[i][:, 1])
    
    # Mean temperature of both slabs
    T_slab_mean = np.mean(np.vstack((T_slab_positive, T_slab_negative)), axis=0)
    return T_slab_positive, T_slab_negative, T_bulk_chunk, T_slab_mean


def pw_lin_exp_const(t, t0, t1, m, c, lam, d):
    lin = m*t + c
    exp_seg = d + (m*t0 + c - d)*np.exp(-lam*(t - t0))
    const = d
    return np.where(t < t0,
                    lin,
                    np.where(t < t1,
                             exp_seg,
                             const))

def slope_one(x, y):
    t0g = x[np.abs(y - y.mean()).argmin()]
    t1g = x[(x > t0g)].mean()
    p0  = [t0g, t1g, 0.0, y[0], 1.0, y[-1]]
    lb  = [x.min(), t0g+1e-6, -np.inf, -np.inf, 1e-6, y.min()]
    ub  = [x.max(), x.max(),   np.inf,  np.inf,  1e3, y.max()]
    popt, pcov, infodict, mesg, ier    = curve_fit(pw_lin_exp_const, x, y, p0=p0,
                        bounds=(lb, ub), max_nfev=5_000, full_output=True)
    t0, t1, m, c, lam, d = popt
    d_endexp = d + (m*t0 + c - d)*np.exp(-lam*(t1 - t0))
    if np.abs(d-d_endexp) > 5e-1:
        raise ValueError("NO")
    if (m*t0 + c - d) > (- m*t0 ):
        raise ValueError("NO")
    return popt[0], popt[2], mesg, ier 


def find_breakpoint_iterative(x, y, min_points=10, maxiter=100, penalty_npoint=0.1, penalty_const=0.1):
    def mse_break(x0):
        idx = np.searchsorted(x, x0, 'right')
        c0 = np.mean(y[:10])
        if idx < min_points:
            idx = min_points
            return np.float64(1e100)
        xi, yi = x[:idx], y[:idx]
        yl = y[idx:]
        m, c = np.polyfit(xi, yi, 1)
        const_par = m*x0 + c0
        return np.sum((yi - (m*xi + c0))**2) + penalty_const*np.sum((yl - const_par)**2)  + 0*np.abs(np.log(len(x)/len(xi)))*penalty_npoint

    res = minimize_scalar(
        mse_break,
        bounds=(x[min_points], x[-1]),
        method='bounded',
        options={'maxiter': maxiter}
    )
    return res.x

# LOAD

In [ ]:
DATA_ALL = {}

In [ ]:
TBulk = False
LOAD = True
h_slab = 9 # Angstrom



# oxid = 0
for oxid in [0, 5, 10 ,20 ,40 ,60 ,80]:
    c_rep = 0
    temps_w = []
    temps_g = []
    pe_g = []
    ke_g = []
    etot = []
    print(f" OXID={oxid:3d} ".center(100, "="))
    for rep in tqdm(range(1,11), desc='loads reps', total=10) :
        try:
            root = Path(f'/media/fabiano/b_dev/graphene_oxide-water/transient/{oxid:d}%')
            base_remote_path = os.path.join(root, f'Transient/graph{oxid:d}%_{rep:d}')
            file_liquid = 'system_h2o.txt'
            file_nanoparticles = 'system_graph.txt'
            file_chunk = 'temp_chunk_bias_1A.out'
            file_slabverify = 'slab_position_verify.txt' 
            file_Pot = 'potential_graph.txt'
            file_Kin = 'kinetic_graph.txt'
            temps_g.append(pd.read_csv(os.path.join(base_remote_path, file_nanoparticles), sep=r'\s+',  skiprows=2, names=["step", f"temp_{c_rep:d}"]))
            if TBulk:
                temps_w.append(pd.read_csv(os.path.join(base_remote_path, file_liquid), sep=r'\s+', skiprows=2, names=["step", f"temp_{c_rep:d}"]))
            elif not LOAD: 
                Tchunks, tot_chunks = read_Tgraph_chunks(base_remote_path, file_chunk, temps_g[0].shape[0])
                T_slab_positive, T_slab_negative, T_bulk_chunk, T_slab_mean = process_chunks(Tchunks,tot_chunks,h_slab)
                df_slab = pd.DataFrame({"step": temps_g[-1]["step"].values, f"temp_{c_rep:d}": T_slab_mean })
                temps_w.append(df_slab)
            pe_g.append(pd.read_csv(os.path.join(base_remote_path, file_Pot), sep=r'\s+', skiprows=2, names=["step", f"pe_{c_rep:d}"]))
            ke_g.append(pd.read_csv(os.path.join(base_remote_path, file_Kin), sep=r'\s+', skiprows=2, names=["step", f"ke_{c_rep:d}"]))  
            etot.append( pd.DataFrame(data={"step": pe_g[-1]["step"], f"etot_{c_rep:d}": pe_g[-1][f"pe_{c_rep:d}"] + ke_g[-1][f"ke_{c_rep:d}"]}) )
            c_rep +=1
        except Exception:
            print("Erro in rep=", rep)
    rep_Tgr = pd.concat(temps_g, axis=1)
    rep_etot = pd.concat(etot, axis=1)
    numpy_dir = os.path.join(f'{root}','numpy')

    if not Path(numpy_dir).exists():
            os.mkdir(numpy_dir)

    if TBulk:
        LOAD = False
    if not LOAD:
        rep_Tw = pd.concat(temps_w, axis=1)
        if not TBulk:
            rep_Tw.to_pickle(os.path.join(numpy_dir,f'rep_Tw_{h_slab}A.pkl'))
            print("saved in: ",os.path.join(numpy_dir,f'rep_Tw_{h_slab}A.pkl'))
    else:
        rep_Tw = pd.read_pickle(os.path.join(numpy_dir,f'rep_Tw_{h_slab}A.pkl'))

    DATA_ALL[oxid] = {
        "Tw": rep_Tw.copy(),
        "Tgr": rep_Tgr.copy(),
        "etot": rep_etot.copy(),
    }
    

# CONCAT

In [ ]:
CONCAT_ALL_DATA = {}
N_rep = 10
for oxid, data in tqdm(DATA_ALL.items(),desc="concat data", total=len(DATA_ALL)):
   
    CONCAT_ALL_DATA[oxid] = {
        "time_s": data["Tw"].iloc[:,0].values * 1e-15, 
        "Tw": np.column_stack([data["Tw"][f"temp_{i}"].values for i in range(N_rep)])[5000:],
        "Tgr": np.column_stack([data["Tgr"][f"temp_{i}"].values for i in range(N_rep)])[5000:],
        "etot": np.column_stack([data["etot"][f"etot_{i}"].values for i in range(N_rep)])[5000:]    
    }
    Nt = CONCAT_ALL_DATA[oxid]["time_s"].shape[0]
    int_t_rep = np.zeros((Nt, N_rep))
    for i in range(N_rep):
        dt_i = data["Tgr"].loc[:, f"temp_{i}"].values - data["Tw"].loc[:, f"temp_{i}"].values
        int_t_rep[1:, i] = cumulative_trapezoid(dt_i, CONCAT_ALL_DATA[oxid]["time_s"])
    CONCAT_ALL_DATA[oxid]['integral'] = int_t_rep[5000:] - int_t_rep[5000]

# $R_K$

In [ ]:
from time import time

In [ ]:
SLOPE_ALL = {}
SLOPE_ensavg_ALL = {}

In [ ]:
for oxid, data in CONCAT_ALL_DATA.items():
    start_s = time()
    print(f" OX STATE {oxid:3d} ".center(120, "="))
    diffT = data['Tgr'] - data['Tw']
    integral = data['integral']
    etot = data['etot']
    scaleT = diffT.std()
    scaleE = etot.std()
    scaleInt = integral.std()
    print("searchin t0 ..", end='')
    N_each = integral.shape[0] // 10
    t0_all = []
    m_all = []
    for i in tqdm(range(10), total=10):
        t0 = find_breakpoint_iterative(integral[:,i]/scaleInt, etot[:,i]/scaleE, min_points=9, maxiter=500, penalty_npoint=2, penalty_const=1)
        integ_temp = integral[:,i]/scaleInt
        if len(integ_temp[integ_temp<t0]) < 10:
            continue
        t0_all.append(t0*scaleInt)
        # m_all.append(m)
    print("done!")
    print(f"\t t0={t0:1.3e}")
    t0_min = np.min(t0_all)

    print(f"t0_min = {t0_min:1.3e}")
    print("linear fit ..", end='')
    x_fit = integral[integral<t0_min]
    y_fit = etot[integral<t0_min]
    x_fit = sm.add_constant(x_fit)  # Adds a constant term to the predictor
    robust_model = sm.OLS(y_fit, x_fit)
    results = robust_model.fit()
    slope = results.params[1]
    slope_reps = []
    all_fit = []
    for i in tqdm(range(10), total=10):
        integral_rep = integral[:,i]
        etot_rep = etot[:,i]
        t0_i = t0_all[i] 
        x_fit = integral_rep[integral_rep<t0_min]
        y_fit = etot_rep[integral_rep<t0_min]
        x_fit = sm.add_constant(x_fit)  # Adds a constant term to the predictor
        robust_model = sm.OLS(y_fit, x_fit)
        results_rep = robust_model.fit()
        all_fit.append(
            results_rep
        )
        slope_reps.append( results_rep.params[1] )

    intercept_reps = [res.params[0] for res in all_fit]
    slope_reps     = [res.params[1] for res in all_fit]
    a_bar = np.mean(intercept_reps)
    m_bar = np.mean(slope_reps)
    x_pred = np.linspace(integral.min(), t0_min, 100)
    y_med = a_bar + m_bar * x_pred
    plt.plot(x_pred, y_med, color='r', zorder =20 , lw=4)

    # slope_err = results.bse[1]
    # slope_err = np.std(slope_reps) * 1.96
    slope = np.mean(slope_reps)

    N_rep = len(slope_reps)          # 10
    slope_se   = np.std(slope_reps, ddof=1) / np.sqrt(N_rep)   # standard error
    slope_err  = 1.96 * slope_se 
    
        
    # }
    SLOPE_ALL[oxid] = {
        "Rk_m" : - AREA[oxid]/ (slope* 4184 / N_A),
        "Rk_e" : AREA[oxid]/ (4184 / N_A) / (slope)**2 * slope_err,
    }
    
    SLOPE_ALL[oxid]["Rk_rse"] = SLOPE_ALL[oxid]["Rk_e"] / abs(SLOPE_ALL[oxid]["Rk_m"])
    SLOPE_ALL[oxid]["Gk_m"] = 1/SLOPE_ALL[oxid]["Rk_m"]
    SLOPE_ALL[oxid]["Gk_e"] = SLOPE_ALL[oxid]["Rk_e"] / SLOPE_ALL[oxid]["Rk_m"]**2
    print(f"\t m = {slope:1.3e} +/- {slope_err:1.3e}")
    print(f"\t Rk = {SLOPE_ALL[oxid]["Rk_m"]:1.3e} +/- {SLOPE_ALL[oxid]["Rk_e"]:1.3e} (i.e )  {SLOPE_ALL[oxid]["Rk_rse"]}")
    print(f"\t Gk = {SLOPE_ALL[oxid]["Gk_m"]:1.3e} +/- {SLOPE_ALL[oxid]["Gk_e"]:1.3e}")
    b = results.params
    x_pred = np.linspace(integral.min(), t0_min, 10)
    pred = results.get_prediction(sm.add_constant(x_pred))
    y_pred = pred.predicted
    iv_l_ols = pred.summary_frame()["mean_ci_lower"]
    iv_u_ols = pred.summary_frame()["mean_ci_upper"]
    print("done!")

    # plot
    indx0  = np.where(integral[:,0]<t0_min)[0]
    int_rep_mean = integral[indx0, :].mean(axis=1)
    etot_mean = etot[indx0, :].mean(axis=1)
    plt.plot(int_rep_mean,etot_mean, color='y')

    slope_ensam_avg = - AREA[oxid]/ (np.polyfit(int_rep_mean,etot_mean,1)[0] * 4184 / N_A)
    print(f"R_ens_avg = {slope_ensam_avg}")
    SLOPE_ensavg_ALL[oxid] = {
        "Rk_m" : - AREA[oxid]/ (slope* 4184 / N_A)  
    }
    SLOPE_ensavg_ALL[oxid]["Gk_m"] = 1/SLOPE_ensavg_ALL[oxid]["Rk_m"]
    n=0
    for res_fint in all_fit:
        indx0 = integral[:, n] < t0_min
        pred = res_fint.get_prediction(sm.add_constant(x_pred))
        y_pred_i = pred.predicted
        plt.plot(x_pred,y_pred_i, color='k', ls=":")
        plt.scatter(integral[indx0,n], etot[indx0,n], s=4, alpha=.3, label = f'Replicate {n}')
        n += 1
    # plt.legend(loc='best', fontsize='small')    
    plt.show()
    # plt.xlim(integral.min(), t0_min)
    end_time =  time()
    print(f"elalpesed time {end_time-start_s} s")

In [ ]:
data_all = [[oxid, data["Gk_m"], data["Gk_e"]] for oxid, data in SLOPE_ALL.items()]
data_all = np.asarray(data_all)

In [ ]:

# plt.errorbar(data_all[:,0], data_all[:,1], yerr=data_all[:,2],)
# plt.scatter(data_all[:,0], data_all[:,1],c='b',marker='o')
x_fit = data_all[:,0]
y_fit = data_all[:,1]
x_fit = sm.add_constant(x_fit)  # Adds a constant term to the predictor
weight = data_all[:,1]**2/data_all[:,2]**2
weight /= weight.mean()
robust_model = sm.GLM(y_fit, x_fit, var_weights=weight)
results = robust_model.fit()
x_pred = np.linspace(data_all[:,0].min(), data_all[:,0].max(), 100)
pred = results.get_prediction(sm.add_constant(x_pred))
y_pred = pred.predicted
iv_l_ols = pred.summary_frame()["mean_ci_lower"]
iv_u_ols = pred.summary_frame()["mean_ci_upper"]
# plt.fill_between(x_pred, iv_l_ols, iv_u_ols, alpha=.3, color='k')
# plt.plot(x_pred, y_pred, color='k')

# ensamble average test

In [ ]:
# test of Gk averaging energy and temperatures among simulations before calculating the slopes 
RUN = False
if RUN:
    data_ensavg_all = [[oxid, data["Gk_m"]] for oxid, data in SLOPE_ensavg_ALL.items()]
    data_all = np.asarray(data_ensavg_all)
    data_all = data_all[:,0:2]
    data_all

    plt.scatter(data_all[:,0], data_all[:,1],c='b',marker='o')
    x_fit = data_all[:,0]
    y_fit = data_all[:,1]
    x_fit = sm.add_constant(x_fit)  # Adds a constant term to the predictor

    robust_model = sm.GLM(y_fit, x_fit)
    results = robust_model.fit()

    x_pred = np.linspace(data_all[:,0].min(), data_all[:,0].max(), 100)
    pred = results.get_prediction(sm.add_constant(x_pred))
    y_pred = pred.predicted
    iv_l_ols = pred.summary_frame()["mean_ci_lower"]
    iv_u_ols = pred.summary_frame()["mean_ci_upper"]
    plt.fill_between(x_pred, iv_l_ols, iv_u_ols, alpha=.3, color='k')
    plt.plot(x_pred, y_pred, color='k')

# Paper Plots

## Gk vs oxid

In [ ]:
scaleup = 1.25
fig = plt.figure(figsize=(3.3*scaleup, 2*scaleup), dpi=250)

with plt.style.context("/home/fabiano/WORK/style.matplot"):
    grid = fig.add_gridspec(1,1, top=.99, right=.99, bottom=.18)
    ax = fig.add_subplot(grid[0] )
    # plt.fill_between(df["time"], df["CA"]-df["std"], df["CA"]+df["std"], alpha=.5)
    plt1 = ax.errorbar(data_all[:,0], data_all[:,1]*1e-8, yerr=data_all[:,2]*1e-8, capsize=2.2, fmt='o', markersize=3.2, zorder=11, color="#282222", lw=.5, label="Instantaneous value")
    plt2 = ax.fill_between(x_pred, iv_l_ols*1e-8, iv_u_ols*1e-8, alpha=.4, zorder=10, color="#5678E7")
    plt3 = ax.plot(x_pred, y_pred*1e-8, '-', lw=1.3, zorder=10, color='#5678E7')
    ax.legend([plt1, (plt2,plt3[0])], ("Simulation Data", "Linear regression (with 95% CB)"))
    ax.set_xlabel("Oxidation (%)")
    
    ax.set_ylabel(r"$\mathregular{1/R_k}$ (x10$^8$ W/m$^2$K)")
fig.savefig("/home/fabiano/graphtmp/plot_article/fig_Gk-oxid.png", dpi=400)
fig.savefig("/home/fabiano/graphtmp/plot_article/fig_Gk-oxid.pdf", dpi=400)

## Slopes (suppy: S1)

In [ ]:
# oxid = int(10)
import matplotlib.cm as cm
import matplotlib.colors as mcolors
from matplotlib import ticker
import seaborn as sns



for oxid, data in CONCAT_ALL_DATA.items():
    print(f" OX STATE {oxid:3d} ".center(120, "="))
    diffT = data['Tgr'] - data['Tw']
    integral = data['integral']
    etot = data['etot']
    scaleT = diffT.std()
    scaleE = etot.std()
    scaleInt = integral.std()
    N_each = integral.shape[0] // 10
    t0_all = []
    m_all = []
    for i in tqdm(range(10), total=10):
        t0 = find_breakpoint_iterative(integral[:,i]/scaleInt, etot[:,i]/scaleE, min_points=9, maxiter=500, penalty_npoint=2, penalty_const=1)
        integ_temp = integral[:,i]/scaleInt
        if len(integ_temp[integ_temp<t0]) < 10:
            continue
        t0_all.append(t0*scaleInt)
    t0_min = np.min(t0_all) 
    print(f"t0_min = {t0_min:1.3e}")
    print("linear fit ..", end='')
    x_fit = integral[integral<t0_min]
    y_fit = etot[integral<t0_min]
    x_fit = sm.add_constant(x_fit)  # Adds a constant term to the predictor
    robust_model = sm.OLS(y_fit, x_fit)
    results = robust_model.fit()
    slope = results.params[1]
    slope_reps = []
    all_fit = []
    for i in tqdm(range(10), total=10):
        integral_rep = integral[:,i]
        etot_rep = etot[:,i]
        t0_i = t0_all[i]
        x_fit = integral_rep[integral_rep<t0_min]
        y_fit = etot_rep[integral_rep<t0_min]
        x_fit = sm.add_constant(x_fit)  # Adds a constant term to the predictor
        robust_model = sm.OLS(y_fit, x_fit)
        results_rep = robust_model.fit()
        all_fit.append(
            results_rep
        )
        slope_reps.append( results_rep.params[1] )
        

    slope = np.mean(slope_reps)

    N_rep = len(slope_reps)          # 10
    slope_se   = np.std(slope_reps, ddof=1) / np.sqrt(N_rep)   # standard error
    slope_err  = 1.96 * slope_se 

        
    # }
    SLOPE_ALL[oxid] = {
        "Rk_m" : - AREA[oxid]/ (slope* 4184 / N_A),
        "Rk_e" : AREA[oxid]/ (4184 / N_A) / (slope)**2 * slope_err,
    }

    SLOPE_ALL[oxid]["Rk_rse"] = SLOPE_ALL[oxid]["Rk_e"] / abs(SLOPE_ALL[oxid]["Rk_m"])
    SLOPE_ALL[oxid]["Gk_m"] = 1/SLOPE_ALL[oxid]["Rk_m"]
    SLOPE_ALL[oxid]["Gk_e"] = SLOPE_ALL[oxid]["Rk_e"] / SLOPE_ALL[oxid]["Rk_m"]**2
    print(f"\t m = {slope:1.3e} +/- {slope_err:1.3e}")
    print(f"\t Rk = {SLOPE_ALL[oxid]["Rk_m"]:1.3e} +/- {SLOPE_ALL[oxid]["Rk_e"]:1.3e}")
    print(f"\t Gk = {SLOPE_ALL[oxid]["Gk_m"]:1.3e} +/- {SLOPE_ALL[oxid]["Gk_e"]:1.3e}")
    b = results.params
    x_pred = np.linspace(integral.min(), t0_min, 10)
    pred = results.get_prediction(sm.add_constant(x_pred))
    y_pred = pred.predicted
    iv_l_ols = pred.summary_frame()["mean_ci_lower"]
    iv_u_ols = pred.summary_frame()["mean_ci_upper"]

    # Plot
    indx0  = np.where(integral[:,0]<t0_min)[0]
    fig = plt.figure(figsize=(3, 1.7), dpi=200)

    with plt.style.context("/home/fabiano/WORK/style.matplot"):
        grid = fig.add_gridspec(1,16,  bottom=.28, left=.23, top=.985, right=.86)
        ax = fig.add_subplot(grid[:-1])
        ax_bar = fig.add_subplot(grid[-1])

        
        # plot mean of linear regressions
        n_fits  =10
        palette = sns.color_palette("blend:#ffe6cc,#e6a1a1,#aa8fcc", n_colors=n_fits)
        cmap    = mcolors.ListedColormap(palette)
        bounds  = np.arange(n_fits + 1)            # [0,1,2,...,n_fits]
        norm    = mcolors.BoundaryNorm(bounds, n_fits)

        # --- a ScalarMappable just for the colorbar ---
        mappable = cm.ScalarMappable(cmap=cmap, norm=norm)
        mappable.set_array([])
                
        intercept_reps = [res.params[0] for res in all_fit]
        slope_reps     = [res.params[1] for res in all_fit]
        a_bar = np.mean(intercept_reps)
        m_bar = np.mean(slope_reps)
        x_pred = np.linspace(integral.min(), t0_min, 100)
        y_med = a_bar + m_bar * x_pred
        
        n=0
        size = .75 
        alpha = 1.0 
        
        for res_fint in all_fit:
            this_color = cmap(n)
            pred = res_fint.get_prediction(sm.add_constant(x_pred))
            y_pred_i = pred.predicted
            mask_n = integral[:, n] < t0_min
            plt0 = ax.plot(x_pred*1e9,y_pred_i*1e-3, color="#000000", ls=":", lw=.8, zorder=100)
            ax.plot(integral[mask_n,n]*1e9, etot[mask_n,n]*1e-3, lw=size, alpha=alpha, color=this_color)
            n += 1
        plt1 = ax.plot(x_pred*1e9, y_med*1e-3, color="#B40000", zorder =120 , lw=1.5)
        ax.set_xlabel(r'$\int\Delta T \, \mathrm{d}t$ $\mathrm{(K\cdot ns)}$')
        ax.set_ylabel("Energy $\mathrm{(Mcal / mol)}$")
        # ax.legend([plt0[0], plt1[0]], ("Single linear fit", "Average linear fit"), loc="lower center", bbox_to_anchor=[0.5, 0.96] )
        cbar = fig.colorbar(mappable,
                    cax=ax_bar,
                    boundaries=bounds,
                    ticks=bounds[:-1] + 0.5,  # centers of each bin
                    spacing='uniform')        # equal‐sized boxes
        cbar.set_ticklabels(np.arange(1, n_fits+1))  
        cbar.set_label('Replica ID')
        cbar.ax.yaxis.set_tick_params('both', width=0.0)
        ax.yaxis.set_major_formatter(ticker.StrMethodFormatter('{x:.1f}'))
        fig.savefig(f"/home/fabiano/graphtmp/plot_article/fig_slopes-oxid{oxid:02d}.png", dpi=400)
        fig.savefig(f"/home/fabiano/graphtmp/plot_article/fig_slopes-oxid{oxid:02d}.pdf", dpi=400)


In [ ]:
import matplotlib.pyplot as plt

# Crea una figura solo con la legenda
fig, ax = plt.subplots(figsize=(3.3, 2), dpi=250)

# Simuliamo le linee per la legenda
linea_media, = ax.plot([], [], color="#B40000", lw=3, label="Ensemble-average OLS fit")
linea_i, = ax.plot([], [], color='#3A445D', ls=":", lw=1, label="Individual OLS fit")
scatter_i = ax.scatter([], [], s=10, alpha=0.4, c='#9E0142', label="")  # un colore qualsiasi dalla palette

# Costruisci la legenda
handles = [linea_media, linea_i, scatter_i]
labels = ["Ensemble-average OLS fit", "Individual OLS fit", ""]  # Terzo label vuoto per non duplicare
ax.legend(handles=handles[:2], labels=labels[:2], loc='center')
ax.axis('off')  # nascondi assi

# Salva o mostra la figura
plt.tight_layout()
plt.savefig("legend_only.pdf", dpi=400)
plt.show()
